In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

In [14]:
def pm25_aqi(Cp):
    IHi = [50, 100, 200, 300, 400, 500]
    ILo = [0, 51, 101, 201, 301, 401]
    BPHi = [30, 60, 90, 120, 250]
    BPLo = [0, 31, 61, 91, 121, 251]

def data_by_date(start_date, end_date, location_id):
    api_url = f"https://api.openaq.org/v2/measurements?date_from={start_date}&date_to={end_date}&limit=7000&sort=desc&parameter=pm25&location_id={location_id}&order_by=datetime"
    response = requests.get(api_url)

    results = response.json()["results"]
    df = pd.DataFrame(columns=['datetime', 'pm2.5'])

    for i, result in enumerate(results):
        datetime = result["date"]["local"]
        pm25 = result["value"]

        df.loc[i] = [datetime, pm25]

    df.datetime = pd.to_datetime(df.datetime)

    df['month'] = df['datetime'].dt.strftime('%m')
    df['day'] = df['datetime'].dt.strftime('%d')
    df['year'] = df['datetime'].dt.strftime('%Y')
    df['time'] = df['datetime'].dt.strftime('%H:%M:%S')

    return df

In [15]:
test = data_by_date('2022-01-01', '2022-02-01', 6928)
test

,datetime,pm2.5,month,day,year,time
0,2022-01-31 14:45:00+05:30,52.00,01,31,2022,14:45:00
1,2022-01-31 13:45:00+05:30,46.00,01,31,2022,13:45:00
2,2022-01-31 12:00:00+05:30,57.00,01,31,2022,12:00:00
3,2022-01-31 11:15:00+05:30,65.88,01,31,2022,11:15:00
4,2022-01-30 10:15:00+05:30,187.51,01,30,2022,10:15:00
5,2022-01-30 09:15:00+05:30,146.53,01,30,2022,09:15:00
6,2022-01-30 08:15:00+05:30,77.20,01,30,2022,08:15:00
7,2022-01-30 07:15:00+05:30,84.28,01,30,2022,07:15:00
8,2022-01-30 06:15:00+05:30,98.91,01,30,2022,06:15:00
9,2022-01-30 05:15:00+05:30,99.70,01,30,2022,05:15:00
